### Serializing LLM Pipelines

Serialization will help us to convert our pipelines to a format that we can save on our disk and load later. It can be saved on the database and also get sent over the network etc.

### Creating a Simple Pipeline
A pipeline that expects a topic from the user, and generates a summary about the topic with `google/flan-t5-large`

In [3]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator

template = """
Please create a summary about the following topic:
{{topic}}
"""

builder = PromptBuilder(template=template)
llm = HuggingFaceLocalGenerator(
    model="google/flan-t5-large", task="text2text-generation", generation_kwargs={"max_new_tokens": 150}
)

pipeline = Pipeline()
pipeline.add_component(name="builder", instance=builder)
pipeline.add_component(name="llm", instance=llm)

pipeline.connect("builder", "llm")

🚅 Components
  - builder: PromptBuilder
  - llm: HuggingFaceLocalGenerator
🛤️ Connections
  - builder.prompt -> llm.prompt (str)

In [6]:
topic = "Climate Change"
result = pipeline.run(data={"builder": {"topic": topic}})
print(result["llm"]["replies"][0])

Climate change is a major threat to the planet.


#### Serialize the Pipeline to YAML

Out of the box, Haystack supports YAML. Use dumps to convert the Pipeline to YAML

In [8]:
yaml_pipeline = pipeline.dumps()

print(yaml_pipeline)

components:
  builder:
    init_parameters:
      required_variables: null
      template: '

        Please create a summary about the following topic:

        {{topic}}

        '
      variables: null
    type: haystack.components.builders.prompt_builder.PromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 150
      huggingface_pipeline_kwargs:
        device: cuda:0
        model: google/flan-t5-large
        task: text2text-generation
      stop_words: null
      streaming_callback: null
      token:
        env_vars:
        - HF_API_TOKEN
        strict: false
        type: env_var
    type: haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator
connections:
- receiver: llm.prompt
  sender: builder.prompt
max_loops_allowed: 100
metadata: {}



### Edit the pipeline  in yaml
Translate to french and run on CPU instead of GPU

In [9]:
# 

yaml_pipeline = """
components:
  builder:
    init_parameters:
      template: "\nPlease translate the following to French: \n{{ sentence }}\n"
    type: haystack.components.builders.prompt_builder.PromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 150
      huggingface_pipeline_kwargs:
        device: cpu
        model: google/flan-t5-large
        task: text2text-generation
        token: null
      stop_words: null
    type: haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator
connections:
- receiver: llm.prompt
  sender: builder.prompt
max_loops_allowed: 100
metadata: {}
"""


### Deserializing a YAML Pipeline back to Python

Use the .loads()

In [10]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator

new_pipeline = Pipeline.loads(yaml_pipeline)
new_pipeline.run(data={
    "builder": {
        "sentence": "Nigeria, a country filled with promising talents!"
    }
})

{'llm': {'replies': ['Nigeria, un pays en pleine créativité!']}}